In [1]:
import numpy as np
import igraph as ig
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from time import time

from common import *
from generate import *
from filter_kruskal import *

In [2]:
common.run(args="--help")

Allowed options:
  -h [ --help ]                      produce help message
  -m [ --mst-kernel ] arg (=1)       Kernel used calculating MST. 0 = regular 
                                     kruskal, 1 = filter-kruskal
  -s [ --sort-kernel ] arg (=0)      Kernel used for sort() [int]
  -p [ --partition-kernel ] arg (=0) Kernel used for partition() [int]
  -f [ --filter-kernel ] arg (=0)    Kernel used for filter() [int]
  -i [ --inputfile ] arg             input file containing graph data. Filepath
                                     must be relative to Ex2/
  -t [ --ouputfile_timings ] arg     Output file for timings. Filepath must be 
                                     relative to Ex2/



In [9]:
args = f"--inputfile input_data/barabasi_50000_10.csv --mst-kernel 0"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv", args=args, print_output=True)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv --inputfile input_data/barabasi_25000_10.csv --mst-kernel 0
Loading /home/hiti/workspace/GPU_ArchComp/Ex2/input_data/barabasi_25000_10.csv

Benchmark results:
Initialize	µ=7.9562e-05s	sigma=0s	total=7.9562e-05	num_calls=1
Kruskal()	µ=3.33075s	sigma=0s	total=3.33075	num_calls=1
grow MST	µ=2.14905s	sigma=0s	total=2.14905	num_calls=1
sort_edgelist()	µ=1.18169s	sigma=0s	total=1.18169	num_calls=1
total	µ=3.33083s	sigma=0s	total=3.33083	num_calls=1


,tag,average,std_dev,total,num_calls
0,Initialize,0.00008,0,0.00008,1
1,Kruskal(),3.33075,0,3.33075,1
2,grow MST,2.14905,0,2.14905,1
3,sort_edgelist(),1.18169,0,1.18169,1
4,total,3.33083,0,3.33083,1


In [10]:
args = f"-i input_data/barabasi_50000_10.csv -m 1"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv",args=args, print_output=False)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv -i input_data/barabasi_25000_10.csv -m 1


,tag,average,std_dev,total,num_calls
0,Initialize,0.000089,0.000000,0.000089,1
1,Kruskal(),0.053563,0.009701,0.803444,15
2,filter(),0.113061,0.069721,1.582850,14
3,grow MST,0.031238,0.008857,0.468570,15
4,partition(),0.000096,0.000034,0.001339,14
5,sort_edgelist(),0.022323,0.004659,0.334848,15
6,total,2.387840,0.000000,2.387840,1
